# Knapsack problem

Given a list of items and a bag maximum capacity, find the best combination of items.

The algorithm to complete:

In [1]:
import random
import math
from functools import reduce

class KnapsackGenetic:
    def __init__(self, params):
        self.ALL_NUMBERS = list(range(params["max_per_item"] + 1))
        self.params = params
        self.specimen = [None] * self.params["generation_size"]

        self.create_initial_population()

    def create_initial_population(self):
        self.specimen = list(map(
            lambda _: list(map(
                lambda _: random.choice(self.ALL_NUMBERS),
                [None] * len(self.params["items"])
            )),
            self.specimen
        ))

    def reduce_specimen(self, total, act):
        cant = act[0]
        total["value"] = total["value"] + act[1].value * cant
        total["weight"] = total["weight"] + act[1].weight * cant
        return total
    
    def fitness(self, specimen):
        # Use params: self.params["max_weight"] to check the max weight of the specimen
        # Use params: self.params["items"]
        
        vals = reduce(self.reduce_specimen, list(zip(specimen, self.params["items"])), {"value":0, "weight":0})
        
        sum = -1
        if vals["weight"] <= self.params["max_weight"]:
            sum = vals["value"]
        return sum

    def is_converged(self):
        if any(self.fitness(specimen) >= self.params["fit_threshold"] for specimen in self.specimen):
            return True
        return False
    
    def get_fit(self):
        evaluations = self.fitness_all()
        print(evaluations)
        max_evaluation = max(evaluations)

        max_index = evaluations.index(max_evaluation)

        return self.specimen[max_index], max_evaluation

    def fitness_all(self):
        return list(map(self.fitness, self.specimen))

    def select_specimen(self, specimen_evaluations):
        specimen_and_evaluations = list(zip(self.specimen, specimen_evaluations))

        specimen_and_evaluations.sort(key=lambda e: e[1], reverse = True)

        n_top = int(math.ceil(len(self.specimen) * params["select_top"]))

        return list(map(lambda s: s[0], specimen_and_evaluations[:n_top]))
  
    def mutate(self, specimen):
        # Use parameter: self.params["max_per_item"] to check maximum for gene
        # Use parameter: self.params["mutation_percentage"]
        n_digits = int(params["mutation_percentage"] * len(specimen) )
        digit_indexes = random.sample(list(range(len(specimen))), n_digits)
        #print("digit_indexes", digit_indexes)
        mutated = specimen[:]
        for idx in digit_indexes:
            mutated[idx] = random.choice(self.ALL_NUMBERS)
        #print(specimen, "---->", mutated)
        return mutated

    def generate_children(self, selected_specimen):  
        mutated_specimen = [None] * len(self.specimen)

        for i in range(len(mutated_specimen)):
            mutated_specimen[i] = self.mutate(random.choice(selected_specimen))

        return mutated_specimen

    def run(self):
        generation_number = 1

        while generation_number <= self.params["max_generations"] and not self.is_converged():
            top_generation = self.get_fit()
            top_str = "".join(str(top_generation[0]))

            print(f"Generation #{generation_number}:\t{top_str}\t{top_generation[1]}")

            specimen_evaluations = self.fitness_all()
            selected_specimen = self.select_specimen(specimen_evaluations)

            self.specimen = self.generate_children(selected_specimen)

            generation_number += 1

        return self.get_fit()

To run the algorithm:

In [14]:
class Item:
    def __init__(self, value, weight):
        self.value = value
        self.weight = weight
    def __repr__(self):
        return "{value:" + str(self.value) +", weight:" + str(self.weight)+"} "

params = {
    "mutation_percentage": 0.5,
    "select_top": 0.1,
    "generation_size": 50,
    "fit_threshold": 36,
    "max_generations": 1000,
    "max_weight": 15,
    "max_per_item": 4,
    "items": [Item(4, 12), Item(2, 2), Item(2, 1), Item(1, 1), Item(10,4)]
}

knapsack = KnapsackGenetic(params)
knapsack.run()



spe = knapsack.get_fit()[0]
infospe = reduce(knapsack.reduce_specimen, list(zip(spe, knapsack.params["items"])), {"value":0, "weight":0})
print("\n\n____________________________\n\nSELECTED:")
print("Specimen: ", spe, "| totals: ", infospe)
print("ITEMS:")
print(knapsack.params["items"])


[-1, -1, -1, -1, 4, -1, -1, -1, -1, -1, -1, 4, 6, -1, 16, 8, -1, -1, -1, -1, -1, -1, -1, 12, 26, -1, -1, 18, -1, -1, -1, -1, -1, -1, 12, -1, -1, -1, 31, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
Generation #1:	[0, 0, 0, 1, 3]	31
[9, 9, 22, -1, -1, -1, -1, 14, 6, -1, 13, 9, -1, -1, -1, -1, 26, 20, -1, 3, -1, -1, 22, -1, 12, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 22, 24, 21, -1, -1, 12, -1, -1, -1, 10, -1, -1, -1]
Generation #2:	[0, 2, 0, 2, 2]	26
[-1, -1, -1, -1, -1, -1, -1, 26, 23, -1, -1, -1, 12, -1, -1, 21, -1, 24, -1, -1, -1, -1, -1, 22, -1, 22, 15, -1, -1, 15, -1, -1, -1, -1, -1, -1, -1, 14, 14, -1, 24, 23, 16, -1, -1, 28, -1, -1, 12, 12]
Generation #3:	[0, 0, 3, 2, 2]	28
[-1, 28, 6, -1, 22, 21, -1, -1, 34, -1, -1, 28, -1, 4, 30, 21, -1, -1, -1, -1, -1, 10, -1, 12, 24, 30, -1, 21, -1, -1, 22, -1, -1, -1, 20, -1, -1, 14, 26, 26, -1, 28, -1, -1, 28, -1, -1, -1, -1, -1]
Generation #4:	[0, 0, 1, 2, 3]	34
[14, -1, -1, -1, -1, -1, 26, -1, -1, -1, -1, -1, -1, 24, -1, -1, 20, 22